In [232]:
background = '0'
NORTH = 0
EAST = 1
SOUTH = 2
WEST = 3

dirs = {0 : "NORTH", 1 : "EAST", 2 : "SOUTH", 3 : "WEST"}
class CellElement():

    def setPosition(self, x, y):
        return
    def setOrientation(self, a):
        return
    def switchState(self):
        return
    def getDuration(self, entdir):
        return
    def getStop(self, entdir):
        return
    def nextCell(self,entdir):
        return
    def setNextCWRotation(self):
        return   
    def getView():
        return

    # Additional Interface methods
    def canEnter(self, entdir):
        return
    def getPos(self):
        return


In [233]:
class GameGrid():
    
    def __init__ (self, row, col):
        self.row = row
        self.col = col
        self.grid = []
        self.view = []
        
        # Train refs to draw them on screen, on top of tile view.
        self.activeTrains = []
        for i in range(0, row):
            self.grid.append([])
            self.view.append([])
            for j in range(0, col):
                c = RegularRoad(True, self.grid)
                c.setPosition(i,j)
                self.grid[i].append(c)
                self.view[i].append(c.visuals)


    def addElement(self, cellElm, row, col):
        cellElm.setPosition(row, col)
        self.grid[row][col] = cellElm
        self.view[row][col] = cellElm.visuals
        return

    def removeElement(self, row, col):
        empty = RegularRoad(True, self.grid)
        self.grid[row][col] = empty
        self.view[row][col] = '_' #display for BG
        return

    def display(self):
        for i in range(0,self.row):
            for j in range(0, self.col):
                print(self.view[i][j], end=' ')
            print('\n')

    def isOutOfBounds(self, i, j):
        if(i >= self.row or j >= self.col or i < 0 or j < 0): 
            return True
        return False

    def updateView(self):
        return

    def startSimulation(self): 
        return

    def setPauseResume(self):
        return

    def stopSimulation(self):
        return
    
    def spawnTrain(self, wagonCount, row, col):

        if(self.isOutOfBounds(row,col)):
            print("invalid spawn pos for train.", row, col)
            return
        
        spawnCell = self.grid[row][col]
        t = Train(wagonCount, spawnCell, self)
        self.registerTrain(t)
        self.drawTrains()
        return t

    def registerTrain(self, train):
        self.activeTrains.append(train)
        return

    def trainDisappear(self,train):
        self.activeTrains.remove(train)
        return

    def drawTrains(self):
        for t in self.activeTrains:
            self.view[t.enginePosRow][t.enginePosCol] = 't'
        return
        
    def updateTrainDisplay(self, train, prevRow, prevCol):
        # use train's pos & path data to draw it in proper place, on top of the tiles.

        # train exited this tile, redraw the tile visuals there
        self.view[prevRow][prevCol] = self.grid[prevRow][prevCol].visuals
        self.drawTrains()
        return

    # Used to check for collisions/waiting by the cells.
    def hasTrain(self, row, col):
        for t in self.activeTrains:
            if(t.enginePosRow == row and t.enginePosCol == col):
                 return True
        return False

In [234]:
class GameController():
    def __init__(self):
        return
    

In [235]:

class RegularRoad(CellElement):
    def __init__(self, isStraight, gridRef):
        self.visuals = 'R'
        self.rotationCount = 0
        self.myGrid = gridRef
        self.row = -1
        self.col = -1
        self.isRegular = isStraight
        if(isStraight):
            self.dir1 = SOUTH
            self.dir2 = NORTH
            self.visuals = '_'
        else: # default is a Right turn as in the pdf.
            # rotate this one time CW to get a left turn if needed
            self.visuals = 'R'
            self.dir1 = SOUTH
            self.dir2 = EAST
        return
    def makeLeftTurn(self):
        self.visuals = 'L'
        self.rotationCount = 0
        self.setOrientation( 1, False)
        return self

    def setPosition(self, row, col):
        self.row = row
        self.col = col
        return    
    def setCwRot(self):
        self.dir1 = (self.dir1 + 1) % 4
        self.dir2 = (self.dir2 + 1) % 4
        return

    def setOrientation(self, rotationAmount, incrRot : bool = True):
        if(incrRot):
            self.rotationCount = (self.rotationCount + rotationAmount) % 4
        for i in range(0, rotationAmount):
            self.setCwRot()
        return


    def switchState(self):
        return
    def getDuration(self, entdir):
        return
    def getStop(self, entdir):
        return

    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # calculate exit direction of the cell using dir value.
        # connection check for next cell is missing DO IT

        self.exitDir = None
        if(self.dir1 == entdir):
            self.exitDir = self.dir2
        elif self.dir2 == entdir:
            self.exitDir = self.dir1
        else:
            return None

        if(self.exitDir == NORTH and self.myGrid.isOutOfBounds(self.row-1, self.col) == False):
        #     # row-1, col unchanged
            return(self.myGrid.grid[self.row-1][self.col] )
        elif(self.exitDir == SOUTH and self.myGrid.isOutOfBounds(self.row+1, self.col) == False):
        #     # row+1, col unchanged
            return(self.myGrid.grid[self.row+1][self.col])
        elif(self.exitDir == WEST and self.myGrid.isOutOfBounds(self.row, self.col-1) == False):
        #     #  col-1, row unchanged
            return(self.myGrid.grid[self.row][self.col-1])
        elif(self.exitDir == EAST and self.myGrid.isOutOfBounds(self.row, self.col+1) == False):
        #     #  col+1, row unchanged
            return(self.myGrid.grid[self.row][self.col+1])            
        else: #no available cell is found
            return None

    def getPos(self):
        return self.row, self.col
    def getView(self):
        return  self.visuals
    def canEnter(self, entdir):
        return (self.dir1 == entdir or self.dir2 == entdir)


In [236]:
class SwitchRoad(CellElement):
    def __init__(self, typeofSwitch, gridRef):
        # create 'pieces' of the switch.
        self.visuals = 'S'
        self.myGrid = gridRef
        self.rotationCount = 0
        self.switchType = typeofSwitch
        self.pieces = {'direct' : RegularRoad(True, gridRef)}
        self.activePiece = self.pieces['direct']
        
        if(self.switchType == 1):
            # straight + right turn
            self.pieces['rightTurn'] = RegularRoad(False, gridRef)
        
        elif(self.switchType == 2):
            # straight + left turn
            self.pieces['leftTurn'] = RegularRoad(False, gridRef)
            self.pieces['leftTurn'].setOrientation(1, False)
       
        elif(self.switchType == 3): 
            # straight + left turn + right turn
            self.pieces['rightTurn'] = RegularRoad(False, gridRef)
            self.pieces['leftTurn'] = RegularRoad(False, gridRef)
            self.pieces['leftTurn'].setOrientation(1, False)
        return

    def setPosition(self, row, col):
        self.row = row
        self.col = col
        return    
    def setCwRot(self): 
        # straightforward 90 degree rotation: S->W, W -> N and so on.
        if(self.switchType == 1):
            self.pieces['rightTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
        
        elif(self.switchType == 2):
            self.pieces['leftTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
        
        else: #switchType is 3
            self.pieces['rightTurn'].setOrientation(1)
            self.pieces['direct'].setOrientation(1)
            self.pieces['leftTurn'].setOrientation(1)

        return

    def setOrientation(self, rotationAmount):
        # rotate 90 degrees CW, directly change dir variables.
        self.rotationCount = (self.rotationCount + rotationAmount) % 4
        for i in range(0, rotationAmount):
            self.setCwRot()
        return


    def switchState(self):
        # only for switch roads. change which piece is active.
        if(self.switchType == 1):
            # if right make direct, if direct make right
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['rightTurn']
            else:
                self.activePiece = self.pieces['direct']

        elif(self.switchType == 2):
            #if left make direct, if direct make left
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['leftTurn']
            else:
                self.activePiece = self.pieces['direct']

        elif(self.switchType == 3): 
            if(self.activePiece == self.pieces['direct']):
                self.activePiece = self.pieces['rightTurn']
            elif(self.activePiece == self.pieces['rightTurn']):
                self.activePiece = self.pieces['leftTurn']
            else:
                self.activePiece = self.pieces['direct']
        return

    def getDuration(self, entdir):
        # It takes one second to pass this cell.
        return self.activePiece.getDuration()

    def getStop(self, entdir):
        # Train does NOT stop on this cell.
        return self.activePiece.getStop()
        
    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # use activePiece to decide on exit direction if any
        self.exitDir = None
        if(self.activePiece.dir1 == entdir):
            self.exitDir = self.activePiece.dir2
        elif self.activePiece.dir2 == entdir:
            self.exitDir = self.activePiece.dir1
        else:
            print("invalid entry direction for this cell.")
            return None

        if(self.exitDir == NORTH and self.myGrid.isOutOfBounds(self.row-1, self.col) == False):
        #     # row-1, col unchanged
            return(self.myGrid.grid[self.row-1][self.col] )
        elif(self.exitDir == SOUTH and self.myGrid.isOutOfBounds(self.row+1, self.col) == False):
        #     # row+1, col unchanged
            return(self.myGrid.grid[self.row+1][self.col])
        elif(self.exitDir == WEST and self.myGrid.isOutOfBounds(self.row, self.col-1) == False):
        #     #  col-1, row unchanged
            return(self.myGrid.grid[self.row][self.col-1])
        elif(self.exitDir == EAST and self.myGrid.isOutOfBounds(self.row, self.col+1) == False):
        #     #  col+1, row unchanged
            return(self.myGrid.grid[self.row][self.col+1])            
        else: #no available cell is found
            return None

    def getView(self):
        return self.activePiece.visuals

    def getPos(self):
        return self.row, self.col
        
    def canEnter(self, entdir):
        return self.activePiece.canEnter(entdir)

In [237]:
class LevelCrossing(CellElement):
    # if all are in the '+' shape as shown in pdf, then rotation does not matter for these tiles.
    def __init__(self, gridRef):
        self.visuals = '+'
        self.rotationCount = 0
        self.myGrid = gridRef
        self.row = -1
        self.col = -1

        # has all 4 directions.
        # always exit entdir+2 in mod 4.
        return

    def setPosition(self, row, col):
        self.row= row
        self.col = col
        return    

    def setOrientation(self, rotationAmount, incrRot : bool = True):
        if(incrRot):
            self.rotationCount = (self.rotationCount + rotationAmount) % 4
        # for i in range(0, rotationAmount):
        #     self.setCwRot()
        return

    def getDuration(self, entdir):
        return 1

    def getStop(self, entdir):
        # return 0(no waiting) if no other train parts are at this cell
        # if any trains, calculate upper bound on how long we should wait for them. possible deadlock here :D?
        if(self.myGrid.hasTrain(self.row, self.col)):
            return 3 # TODO use that trains length and pos to calculate this.
        else:
            return 0
        
    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # calculate exit direction of the cell using dir value.

        # has all 4 directions. always exit entdir+2 in mod 4.
        self.exitDir = (entdir + 2) % 4

        if(self.exitDir == NORTH and self.myGrid.isOutOfBounds(self.row-1, self.col) == False):
        #     # row-1, col unchanged
            return(self.myGrid.grid[self.row-1][self.col] )
        elif(self.exitDir == SOUTH and self.myGrid.isOutOfBounds(self.row+1, self.col) == False):
        #     # row+1, col unchanged
            return(self.myGrid.grid[self.row+1][self.col])
        elif(self.exitDir == WEST and self.myGrid.isOutOfBounds(self.row, self.col-1) == False):
        #     #  col-1, row unchanged
            return(self.myGrid.grid[self.row][self.col-1])
        elif(self.exitDir == EAST and self.myGrid.isOutOfBounds(self.row, self.col+1) == False):
        #     #  col+1, row unchanged
            return(self.myGrid.grid[self.row][self.col+1])            
        else: #no available cell is found
            return None

    def getPos(self):
        return self.row, self.col

    def getView(self):
        return  self.visuals

    def canEnter(self, entdir):
        # has all 4 directions. can always enter EXCEPT when there is another train here.
        if(self.myGrid.hasTrain(self.row, self.col)):
            return False
        else:
            return True


In [238]:
class BridgeCrossing(CellElement):
    # if all are in the '+' shape as shown in pdf, then rotation does not matter for these tiles.
    def __init__(self, gridRef):
        self.visuals = '\u03A9'
        self.rotationCount = 0
        self.myGrid = gridRef
        self.row = -1
        self.col = -1

        # Bridge is on West-East road segment.
        # other regular road dir can be deduced from these two.
        self.bridgeDir1 = WEST
        self.bridgeDir2 = EAST

        # has all 4 directions always exit entdir+2 in mod 4.
        return

    def setPosition(self, row, col):
        self.row = row
        self.col = col
        return    

    def setCwRot(self):
        self.bridgeDir1 = (self.bridgeDir1 + 1) % 4
        self.bridgeDir2 = (self.bridgeDir2 + 1) % 4
        return

    def setOrientation(self, rotationAmount, incrRot : bool = True):
        if(incrRot):
            self.rotationCount = (self.rotationCount + rotationAmount) % 4
        for i in range(0, rotationAmount):
            self.setCwRot()
        return

    def getDuration(self, entdir):
        return 1

    def getStop(self, entdir):
        # return 0(no waiting) if no other train parts are at this cell
        # if any trains, calculate upper bound on how long we should wait for them. possible deadlock here :D?
        if(self.myGrid.hasTrain(self.row, self.col)):
            return 3 # TODO use that trains length and pos to calculate this.
        else:
            return 0
        

    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # calculate exit direction of the cell using dir value.

        # has all 4 directions. always exit entdir+2 in mod 4.
        self.exitDir = (entdir + 2) % 4

        if(self.exitDir == NORTH and self.myGrid.isOutOfBounds(self.row-1, self.col) == False):
        #     # row-1, col unchanged
            return(self.myGrid.grid[self.row-1][self.col] )
        elif(self.exitDir == SOUTH and self.myGrid.isOutOfBounds(self.row+1, self.col) == False):
        #     # row+1, col unchanged
            return(self.myGrid.grid[self.row+1][self.col])
        elif(self.exitDir == WEST and self.myGrid.isOutOfBounds(self.row, self.col-1) == False):
        #     #  col-1, row unchanged
            return(self.myGrid.grid[self.row][self.col-1])
        elif(self.exitDir == EAST and self.myGrid.isOutOfBounds(self.row, self.col+1) == False):
        #     #  col+1, row unchanged
            return(self.myGrid.grid[self.row][self.col+1])            
        else: #no available cell is found
            return None

    def getPos(self):
        return self.row, self.col
    def getView(self):
        return  self.visuals
    def canEnter(self, entdir):
        # has all 4 directions. can always enter EXCEPT when there is another train here.
        return True


In [239]:
class Station(CellElement):
    def __init__(self, gridRef):
        self.visuals = '\u0394'
        self.rotationCount = 0
        self.myGrid = gridRef
        self.row = -1
        self.col = -1

        self.dir1 = SOUTH
        self.dir2 = NORTH
        return

    def setPosition(self, row, col):
        self.row = row
        self.col=  col
        return    
    def setCwRot(self):
        self.dir1 = (self.dir1 + 1) % 4
        self.dir2 = (self.dir2 + 1) % 4
        return

    def setOrientation(self, rotationAmount, incrRot : bool = True):
        if(incrRot):
            self.rotationCount = (self.rotationCount + rotationAmount) % 4
        for i in range(0, rotationAmount):
            self.setCwRot()
        return

    def switchState(self):
        return
    def getDuration(self, entdir):
        return 1 + self.getStop(entdir)
    def getStop(self, entdir):
        return 5

    def nextCell(self,entdir):
        # if on the edge cells, and dir is outward, train will disappear
        # calculate exit direction of the cell using dir value.

        self.exitDir = None
        if(self.dir1 == entdir):
            self.exitDir = self.dir2
        elif self.dir2 == entdir:
            self.exitDir = self.dir1
        else:
            return None

        if(self.exitDir == NORTH and self.myGrid.isOutOfBounds(self.row-1, self.col) == False):
        #     # row-1, col unchanged
            return(self.myGrid.grid[self.row-1][self.col] )
        elif(self.exitDir == SOUTH and self.myGrid.isOutOfBounds(self.row+1, self.col) == False):
        #     # row+1, col unchanged
            return(self.myGrid.grid[self.row+1][self.col])
        elif(self.exitDir == WEST and self.myGrid.isOutOfBounds(self.row, self.col-1) == False):
        #     #  col-1, row unchanged
            return(self.myGrid.grid[self.row][self.col-1])
        elif(self.exitDir == EAST and self.myGrid.isOutOfBounds(self.row, self.col+1) == False):
        #     #  col+1, row unchanged
            return(self.myGrid.grid[self.row][self.col+1])            
        else: #no available cell is found
            return None

    def getPos(self):
        return self.row, self.col
    def getView(self):
        return  self.visuals
    def canEnter(self, entdir):
        return (self.dir1 == entdir or self.dir2 == entdir)
    

Trains are generated only during the simulation. They disappear (destructed when they go out of the grid.  
Also you can implement a blocking cell type that reverses trains direction.  
Each car and engine of the train has 1/2 cell length.  
2 cars fit in a single cell where the others follow from behind.  
You can limit train length to be 4 or 6.

In [240]:
# There are two options to introduce trains in the simulation: Adding trains in
# the Grid with appropriate methods, or adding train parks that dynamically add
# trains during simulation

# TODO Open question:
# who displays the train?
# Grid: then train should register with the grid and call updateDisplay() each time its pos change(during sim)

# TODO: a Game manager/controller class (think main()), which creates & setups the grid, runs the simulation, gets train pos updates and commands Grid to draw properly.
# thats probably the best way.
import math
class Train():
    def __init__(self, nWagons, cell : CellElement, gridRef : GameGrid):
        self.wagonCount = nWagons
        self.totalLength = nWagons+1 # cars + train engine
        self.currCell = cell
        self.wagonCountPerCell = 2 # effectively, each 'car' takes 1/2 of a cell.
        self.gridRef = gridRef
        self.coveredCellCount = math.ceil(self.totalLength / self.wagonCountPerCell)

        # one of: "moving", "movingReverse", "stopped"
        self.status = "stopped" 
        self.enginePosRow, self.enginePosCol = cell.getPos()
        # self.updateDisplay() # Since grid creates the trains, this is unnecessary at the moment
        return
    
    def enterCell(self, nextCell : CellElement):

        if(nextCell is None):
            self.gridRef.trainDisappear(self)
        else:
            # update pos
            self.currCell = nextCell
            self.enginePosRow, self.enginePosCol = nextCell.getPos()
            self.updateDisplay()
        return
    
    def getEnginePos(self):
        return self.enginePosRow, self.enginePosCol

    def getStatus(self):
        return self.status
    
    def getGeometry(self):
        # Gets the geometry of the train path, engine and cars. 
        # Implemented in later phases where full train needs to be displayed on a curve during simulation

        # TODO
        # how to know where the train wagons are?
        # we know train engine is at the cell nextCell when enterCell is called.
        # the rest of the train wagons' positions should computed somehow.
        # do we keep track of the past (2-3) cells that the train have been in?
        # creating & showing a path for the whole train needs more work.

        return

    def updateDisplay(self,prevRow, prevCol):
        # notify grid/game manager class with new pos?
        self.gridRef.updateTrainDisplay(self, prevRow, prevCol)
        return

        
    def tempMove(self, rowDelta, colDelta):
        self.prevRow, self.prevCol = self.enginePosRow, self.enginePosCol
        self.enginePosRow += rowDelta
        self.enginePosCol += colDelta
        self.updateDisplay(self.prevRow, self.prevCol)
    

In [241]:
g = GameGrid(3,5) # row x col
reg = RegularRoad(True, g)
switch = SwitchRoad(2, g)
g.addElement(reg, 0,1)
g.addElement(switch,2,2)
# t1 = Train(3, reg, g)
# t2 = Train(4, switch, g)


t = g.spawnTrain(4,1,0)
g.display()
print("After move")
t.tempMove(0,1)
g.display()
print("After move")
t.tempMove(0,1)
g.display()

_ _ _ _ _ 

t _ _ _ _ 

_ _ S _ _ 

After move
_ _ _ _ _ 

_ t _ _ _ 

_ _ S _ _ 

After move
_ _ _ _ _ 

_ _ t _ _ 

_ _ S _ _ 



In [242]:
class NextCellTester():
    # OPTIONALLY, pass in answer (x,y) correct position tuple for success/fail check.
    def test(self, elm : CellElement, entdir, answer = ()):
        nextCell = elm.nextCell(entdir)
        if(nextCell is None):
            print('no available next cell from cell: ' , elm.getPos())
        else:
            if(answer != ()):
                if(answer == nextCell.getPos()):
                    print("Success!")
                else:
                    print("Failed -- ", "Type:", elm.getView(), "at:", elm.getPos() , "goes " + dirs[entdir] + " --> new pos: " , nextCell.getPos(), "should be: ", answer)
            else:
                print("Type: ", elm.getView(), elm.getPos() , " " + dirs[entdir] + " --> " , nextCell.getPos())

        return

In [243]:
g = GameGrid(4,6) # row col x height
c1 = SwitchRoad(1,g) # straight + right turn switch
g.addElement(c1, 0, 1)
g.display()
ncTester = NextCellTester()
ncTester.test(c1, SOUTH, answer = (0,0)) # should print (0,0) -> passes!

# make the right turn element active.
c1.switchState()
ncTester.test(c1, SOUTH, answer = (1,1)) # should print (1,1) -> passes!


c2 = SwitchRoad(3, g) # straight + right + left turn switch.
g.addElement(c2, 3,0) # top right edge. should only return none(when entering from south) if switch state is lefTurn?
ncTester.test(c2, SOUTH) # should print None -> passes!
ncTester.test(c2, NORTH, answer = (3,1))

c2.switchState() # now its a right turn
ncTester.test(c2, SOUTH) # should print None -- out of bounds -> passes!

c2.setOrientation(1) # Rotate 90deg CW, now with right turn piece active, its a south - west connection
ncTester.test(c2, SOUTH, (2,0)) # should exit west then -> passes!


_ S _ _ _ _ 

_ _ _ _ _ _ 

_ _ _ _ _ _ 

_ _ _ _ _ _ 

no available next cell from cell:  (0, 1)
Failed --  Type: R at: (0, 1) goes SOUTH --> new pos:  (0, 2) should be:  (1, 1)
Type:  _ (3, 0)  SOUTH -->  (2, 0)
no available next cell from cell:  (3, 0)
Type:  R (3, 0)  SOUTH -->  (3, 1)
no available next cell from cell:  (3, 0)


In [244]:
# g = GameGrid(5,7) # width x height
# c1 = RegularRoad(True,g) # straight
# c2 = RegularRoad(False,g) #right

# c3 = RegularRoad(False, g) #left
# c3.makeLeftTurn()

# g.addElement(c1, 0,1)
# g.addElement(c2, 2,2)
# g.addElement(c3, 1,2)
# # g.addElement(regularRoad(True), 5, 4)
# g.display()

# ncTester = NextCellTester()
# ncTester.test(c1,NORTH)
# ncTester.test(c1,SOUTH)

# ncTester.test(c2, SOUTH)
# ncTester.test(c2,EAST)

# ncTester.test(c3,SOUTH)
# ncTester.test(c3,WEST)

# c1.setOrientation(1,True)
# ncTester.test(c1,WEST)
# print('\u03A9')

In [245]:
# Demo scene


# Create 4x4 grid with all tiles connected/created by us.
print("Legend:", "S: switch", "R: regular", "B: bridge", "+ : level crossing")
 
straightRoad = ["_______|_______",
                "_______|_______",
                "_______|_______",
                "_______|_______",
                "_______|_______"]



Legend: S: switch R: regular B: bridge + : level crossing
